In [3]:
import pandas as pd
import numpy as np
from sklearn import datasets
import sklearn

feature = pd.read_csv('Feature_Dataset.csv')
shuffled = sklearn.utils.shuffle(feature)

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(shuffled, test_size=0.2)

In [9]:
y_train = train.iloc[:,:7]
x_train = train.iloc[:,7:]